<a href="https://colab.research.google.com/github/anjaleeDS/Sandbox/blob/master/spacy_ner_satanic_bible.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
from spacy import displacy
import pandas as pd
import altair as alt

# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive
%cd My\ Drive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive
/gdrive/My Drive


In [3]:
myfile = 'TSB.txt'
f = open(myfile, "rb")
text_read = f.read().decode(errors='replace')
# text_read = text.encode('ascii',errors='ignore')
text = str(text_read)
text = text.replace('\t','')
text = text.replace('\n','')
text = text.replace('\r','')

In [4]:
# print((text))
!python -m spacy download en_core_web_lg

# STOP AND RESTART RUNTIME


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [5]:
nlp = spacy.load('en_core_web_lg')
doc = nlp(text)
displacy.render(doc, style='ent', jupyter=True)


In [ ]:
list_text = []
list_ent = []

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
    list_text.append(ent.text)
    list_ent.append(ent.label_)


In [63]:
text_df = pd.DataFrame(list_text, columns=['text'])
text_df['ent'] = list_ent
text_df['cat'] = text_df.text+text_df.ent

text_df

,text,ent,cat
0,Satanic,ORG,SatanicORG
1,Anton Szandor LaVey,PERSON,Anton Szandor LaVeyPERSON
2,The Black Pope,WORK_OF_ART,The Black PopeWORK_OF_ART
3,Anton LaVey,PERSON,Anton LaVeyPERSON
4,only 16 years old,DATE,only 16 years oldDATE
...,...,...,...
3986,American,NORP,AmericanNORP
3987,Avon,ORG,AvonORG
3988,first,ORDINAL,firstORDINAL
3989,Richard F. Burton,PERSON,Richard F. BurtonPERSON


In [64]:
#interested in the top N entities involved
def top_ents(entity, data_df, num):
  type_df = data_df[data_df.ent==entity]
  return(type_df.text.value_counts().head(num))

#what are the entities available?
text_df.ent.value_counts()

PERSON         1125
ORG             753
DATE            482
CARDINAL        342
NORP            340
GPE             319
WORK_OF_ART     202
ORDINAL         142
LOC             101
PRODUCT          49
TIME             47
LANGUAGE         36
FAC              21
EVENT            11
MONEY            11
LAW               6
QUANTITY          3
PERCENT           1
Name: ent, dtype: int64

In [70]:
#use the custom function and graph it!
temp = top_ents('PRODUCT', text_df, 10)

#make into df for graphing
temp_df = temp.to_frame(name='count').reset_index()

bars = alt.Chart(temp_df).mark_bar().encode(
    x='count:Q',
    y=alt.Y('index:N', sort='x'),
    tooltip=('index:N', 'count:Q')
).interactive()

bars.properties(height=900)

alt.Chart(...)

In [ ]:
#graph it ALL
bars = alt.Chart(text_df).mark_bar().encode(
    x='count(ent):Q',
    y=alt.Y('text:N', sort='x'),
    tooltip=('ent:N', 'count(ent):Q')
).interactive()


text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='ent:Q'
)

bars.properties(height=900)